In [1]:
import torch # Delo z matriki z CPU/GPU podporo
from torchvision import datasets, transforms # Predprocesiranje slik
from torch.utils.data import DataLoader # Obravnava podatkov po manjših sklopih

In [14]:
data_dir = '../Data/Podatki_split'

In [15]:
# Definicija transofrmacije za konsistentnost podatkov (CNN želi 224x224, normalizacija)
transform = transforms.Compose([
    transforms.Resize((224, 224)),
    transforms.ToTensor(),
    transforms.Normalize([0.5, 0.5, 0.5], [0.5, 0.5, 0.5])
])

In [16]:
# Naloži slike iz map
batch_size = 32

train_dataset = datasets.ImageFolder(root=f'{data_dir}/timeofday/train', transform=transform)
val_dataset   = datasets.ImageFolder(root=f'{data_dir}/timeofday/val', transform=transform)
test_dataset  = datasets.ImageFolder(root=f'{data_dir}/timeofday/test', transform=transform)

In [21]:
# Razdeli slike po sklopih, boljša učinkovitost
train_loader = DataLoader(train_dataset, batch_size=batch_size, shuffle=True)
val_loader = DataLoader(val_dataset, batch_size=batch_size)
test_loader = DataLoader(test_dataset, batch_size=batch_size)

In [22]:
print("Oznake/Labels:", train_dataset.classes)
print("Število slik v učnem sklopu:", len(train_dataset))

Oznake/Labels: ['day', 'night']
Število slik v učnem sklopu: 6430


#### Training a Classifier: https://docs.pytorch.org/tutorials/beginner/blitz/cifar10_tutorial.html
##### Docs: https://www.geeksforgeeks.org/introduction-convolution-neural-network/

In [23]:
import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim

In [24]:
class TimeOfDayCNN(nn.Module):
    def __init__(self):
        super(TimeOfDayCNN, self).__init__()

        # Dve konvolucijske plasti
        self.conv1 = nn.Conv2d(in_channels=3, out_channels=16, kernel_size=3, padding=1)
        self.pool = nn.MaxPool2d(2, 2)
        
        self.conv2 = nn.Conv2d(16, 32, 3, padding=1)

        # Popolnoma povezana/Fully connected plast
        self.fc1 = nn.Linear(32 * 56 * 56, 128)
        self.fc2 = nn.Linear(128, 2)
        
    def forward(self, x): # Pooling
        x = self.pool(F.relu(self.conv1(x)))
        x = self.pool(F.relu(self.conv2(x)))
        x = x.view(-1, 32 * 56 * 56)
        x = F.relu(self.fc1(x))
        x = self.fc2(x)
        return x

#### Testiranje obnašanja podatkov v modelu

In [25]:
# device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
device = torch.device("cpu")
model = TimeOfDayCNN().to(device)

dataiter = iter(train_loader)
images, labels = next(dataiter)

print("Oblika vhodnih slik:", images.shape)
print("Oznake:", labels)

images = images.to(device)
outputs = model(images)

print("Oblika izhoda modela:", outputs.shape)  # želim [32, 2] za 32 slik in 2 različne oznake

Oblika vhodnih slik: torch.Size([32, 3, 224, 224])
Oznake: tensor([1, 1, 0, 1, 0, 1, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 1, 1, 1, 0, 0, 1, 0, 1,
        0, 0, 0, 1, 1, 0, 0, 1])
Oblika izhoda modela: torch.Size([32, 2])


In [26]:
# Loss funkcija in optimizer
criterion = nn.CrossEntropyLoss()
optimizer = optim.Adam(model.parameters(), lr=0.001)

#### Treniranje in validacija mreže

In [27]:
def validate(model, validation_loader, criterion):
    model.eval()
    
    running_loss = 0.0
    correct = 0
    total = 0

    with torch.no_grad():
        for inputs, labels in validation_loader:
            inputs, labels = inputs.to(device), labels.to(device)

            outputs = model(inputs)
            loss = criterion(outputs, labels)

            running_loss += loss.item()
            _, predicted = torch.max(outputs, 1)
            total += labels.size(0)
            correct += (predicted == labels).sum().item()

    avg_loss = running_loss / len(validation_loader)
    accuracy = 100 * correct / total

    return avg_loss, accuracy

In [28]:
from tqdm import tqdm

num_epochs = 15 # Ni isto kot epizoda!, epoch - SL, episode - RL

for epoch in range(num_epochs):
    running_loss = 0.0
    model.train()

    progress_bar = tqdm(train_loader, desc=f"Epoch {epoch+1}/{num_epochs}")

    for images, labels in progress_bar:
        images, labels = images.to(device), labels.to(device)

        optimizer.zero_grad()
        outputs = model(images)
        loss = criterion(outputs, labels)
        loss.backward()                  # Tukaj se izvede backpropagation
        optimizer.step()

        running_loss += loss.item()
        progress_bar.set_postfix({"Loss": running_loss / (progress_bar.n + 1)})

    avg_loss = running_loss / len(train_loader)

    # Validacija
    val_loss, val_accuracy = validate(model, val_loader, criterion)
    
    print(f"Epoch {epoch+1}/{num_epochs} - Training Loss: {avg_loss:.4f}, "
          f"Validation Loss: {val_loss:.4f}, Validation Accuracy: {val_accuracy:.2f}%")

Epoch 1/15: 100%|████████████████████████████████████████████████████████| 201/201 [02:25<00:00,  1.38it/s, Loss=0.129]


Epoch 1/15 - Training Loss: 0.1287, Validation Loss: 0.0523, Validation Accuracy: 99.06%


Epoch 2/15: 100%|███████████████████████████████████████████████████████| 201/201 [01:53<00:00,  1.77it/s, Loss=0.0523]


Epoch 2/15 - Training Loss: 0.0523, Validation Loss: 0.0484, Validation Accuracy: 98.91%


Epoch 3/15: 100%|███████████████████████████████████████████████████████| 201/201 [01:56<00:00,  1.73it/s, Loss=0.0418]


...

Epoch 13/15 - Training Loss: 0.0000, Validation Loss: 0.1358, Validation Accuracy: 98.91%


Epoch 14/15: 100%|█████████████████████████████████████████████████████| 201/201 [01:51<00:00,  1.80it/s, Loss=1.66e-5]


Epoch 14/15 - Training Loss: 0.0000, Validation Loss: 0.1383, Validation Accuracy: 98.91%


Epoch 15/15: 100%|█████████████████████████████████████████████████████| 201/201 [01:49<00:00,  1.84it/s, Loss=1.37e-5]


Epoch 15/15 - Training Loss: 0.0000, Validation Loss: 0.1405, Validation Accuracy: 98.91%


In [31]:
torch.save(model.state_dict(), "model_daynight.pth")

#### Testiranje modela

In [ ]:
model = TimeOfDayCNN()
model.load_state_dict(torch.load("model.pth", weights_only=True))
model.eval()

In [ ]:
def test_model(model, device, test_loader):
    model.eval()
    correct = 0
    total = 0

    with torch.no_grad():
        for data, target in test_loader:
            data, target = data.to(device), target.to(device)

            output = model(data)
            pred = output.argmax(dim=1, keepdim=True)

            correct += pred.eq(target.view_as(pred)).sum().item()
            total += target.size(0)

    accuracy = 100. * correct / total
    print(f'Test Accuracy: {accuracy:.2f}%')


In [ ]:
test_model(model, device, test_loader)